In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from processingQst import *
from similarityFcts import *
from tqdm import tqdm
import pandas as pd

In [ ]:
df = pd.read_csv("./input/csv/dev.csv",delimiter=";")

Qst = df["question"]
id  = df["id"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
nbrRep = 5
print(len(Qst))
data_list = []
data_listAux = []
RepListIndx = 'abcde'

In [ ]:
def writeJson(path,data):
    with open(path,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)

In [ ]:
with open('output/corpusMS.json', 'r') as f:
    corpusA = json.load(f)
with open('output/corpusMerck.json', 'r') as fs:
    corpusB = json.load(fs)

In [ ]:

for cpt in tqdm(range(len(Qst))):
    # if (cpt > 146):
    #     break
    if (cpt>=0):
        qst = Qst[cpt]
        nwQst = ''
        listMed = []
        keywordQ = []
        keywordQP = []
        ListMedAns = ''
        Spword = False
        data_dict = {}
 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
        else: 
            WrongQst = False 
        
        qst = del_betParenthese(qst)
        fragQst = splitQst(qst)

        for icpt in range(len(fragQst)):
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                keywordQ.extend(listMed)
        nwQst = nwQst.lstrip()
        nwQstp = ''
        var = ''
        varcpt = 0
        for j in range(nbrRep):
            keywordQP = []
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            keywordQP.extend(RecoverListMed(Ans))
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            
            taux_sim = getRepDB(keywordQ,keywordQP,corpusA,nwQstp)
            taux_simB = getRepDB(keywordQ,keywordQP,corpusB,nwQstp)
            if taux_sim > 0.9 or taux_simB > 0.68:
                if var !="":
                    var = var + '|'
                var = var + RepListIndx[j]
                varcpt +=1

        data_dict = {'id': id[cpt], 'correct_answers': var}
        data_dictAux = {'id': id[cpt], 'nbr_correct_answers': varcpt}
        data_list.append(data_dict)
        data_listAux.append(data_dictAux)



In [ ]:
# ---
df = pd.DataFrame(data_list)
df.to_csv('./output/tachePrincipale.csv', sep=';', index=False)
# ---
dfs = pd.DataFrame(data_listAux)
dfs.to_csv('./output/tacheAnnexe.csv', sep=';', index=False)